# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [1]:
import numpy as np
import pandas as pd
from citipy import citipy
import requests
import time
from datetime import datetime
from config import weather_api_key

In [22]:
NUM_POINTS = 50    # DEBUG 2000
BASE_URL = ("https://api.openweathermap.org/data/2.5/weather?"
            f"appid={weather_api_key}&units=imperial")
OUTPUT_DATA_PATH = "cities.csv"

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [3]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=NUM_POINTS)
lngs = np.random.uniform(low=-180.000, high=180.000, size=NUM_POINTS)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [4]:

# for coordinate in coordinates:
#     print(coordinate[0], coordinate[1])
print(len(coordinates))
print(coordinates[0])
print(coordinates[1])

50
(39.068209717737034, -60.34127521173403)
(-18.577049684067376, 143.32125163022755)


## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [5]:

# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    city
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(len(cities))
print(cities)


44
['halifax', 'atherton', 'nogliki', 'innisfail', 'severo-kurilsk', 'omboue', 'chokurdakh', 'boyolangu', 'port alfred', 'arraial do cabo', 'atuona', 'tuktoyaktuk', 'maricao', 'rikitea', 'upernavik', 'pedernales', 'puerto ayora', 'kapaa', 'nizhneyansk', 'qaqortoq', 'lebu', 'ushuaia', 'albany', 'saldanha', 'mahajanga', 'arcachon', 'luderitz', 'chuy', 'khatanga', 'hilo', 'amot', 'tasiilaq', 'okhotsk', 'saint-francois', 'avarua', 'chicama', 'ipameri', 'bluff', 'gouyave', 'belushya guba', 'dikson', 'north bend', 'dong xoai', 'hofn']


## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [6]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = BASE_URL + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]                
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | halifax
Processing Record 2 of Set 1 | atherton
Processing Record 3 of Set 1 | nogliki
Processing Record 4 of Set 1 | innisfail
Processing Record 5 of Set 1 | severo-kurilsk
Processing Record 6 of Set 1 | omboue
Processing Record 7 of Set 1 | chokurdakh
Processing Record 8 of Set 1 | boyolangu
Processing Record 9 of Set 1 | port alfred
Processing Record 10 of Set 1 | arraial do cabo
Processing Record 11 of Set 1 | atuona
Processing Record 12 of Set 1 | tuktoyaktuk
Processing Record 13 of Set 1 | maricao
Processing Record 14 of Set 1 | rikitea
Processing Record 15 of Set 1 | upernavik
Processing Record 16 of Set 1 | pedernales
Processing Record 17 of Set 1 | puerto ayora
Processing Record 18 of Set 1 | kapaa
Processing Record 19 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 20 of Set 1 | qaqortoq
Processing Record 21 of Set 1 | lebu
Processing Record 22 of Set 1 | ushuaia
Pr

In [7]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)


42

## Add the weather data to a new DataFrame.

In [12]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [13]:
# Display sample data
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Halifax,44.6453,-63.5724,31.21,97,100,20.71,CA,2023-02-14 01:42:00
1,Atherton,53.5237,-2.4935,42.46,87,0,3.00,GB,2023-02-14 01:46:24
2,Nogliki,51.8333,143.1667,-1.21,88,88,14.81,RU,2023-02-14 01:46:25
3,Innisfail,-17.5333,146.0333,79.52,69,100,1.86,AU,2023-02-14 01:46:25
4,Severo-Kurilsk,50.6789,156.1250,26.26,93,100,8.72,RU,2023-02-14 01:46:26
5,Omboue,-1.5746,9.2618,78.01,87,100,4.21,GA,2023-02-14 01:46:26
6,Chokurdakh,70.6333,147.9167,-35.09,99,95,3.69,RU,2023-02-14 01:46:27
7,Boyolangu,-8.1181,111.8935,80.15,81,87,4.23,ID,2023-02-14 01:46:28
8,Port Alfred,-33.5906,26.8910,71.19,97,16,15.30,ZA,2023-02-14 01:46:28
9,Arraial Do Cabo,-22.9661,-42.0278,76.42,91,100,9.48,BR,2023-02-14 01:46:29


In [14]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [15]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Date"]
# Recreate the DataFrame by using the new column order
city_data_df = city_data_df[new_column_order]
# Display sample data
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Halifax,CA,44.6453,-63.5724,31.21,97,100,20.71,2023-02-14 01:42:00
1,Atherton,GB,53.5237,-2.4935,42.46,87,0,3.00,2023-02-14 01:46:24
2,Nogliki,RU,51.8333,143.1667,-1.21,88,88,14.81,2023-02-14 01:46:25
3,Innisfail,AU,-17.5333,146.0333,79.52,69,100,1.86,2023-02-14 01:46:25
4,Severo-Kurilsk,RU,50.6789,156.1250,26.26,93,100,8.72,2023-02-14 01:46:26
5,Omboue,GA,-1.5746,9.2618,78.01,87,100,4.21,2023-02-14 01:46:26
6,Chokurdakh,RU,70.6333,147.9167,-35.09,99,95,3.69,2023-02-14 01:46:27
7,Boyolangu,ID,-8.1181,111.8935,80.15,81,87,4.23,2023-02-14 01:46:28
8,Port Alfred,ZA,-33.5906,26.8910,71.19,97,16,15.30,2023-02-14 01:46:28
9,Arraial Do Cabo,BR,-22.9661,-42.0278,76.42,91,100,9.48,2023-02-14 01:46:29


In [20]:
# Display the datatypes of each colunm by accessing the dtypes DataFrame property
city_data_df.dtypes

City           object
Country        object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Date           object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [23]:
# Export the city_data DataFrame into a CSV file
city_data_df.to_csv(OUTPUT_DATA_PATH, index_label="City_ID")